In [1]:
"""5가지 요구사항 : 

1. 웹크롤링을 통하여 관심주제에 대한 감정분석 데이터를 수집하고 수집한 내용과 코드 설명을 병행하여 제시하세요

   *참조 : 7주차 실습 코드 3. 종합실습 코드를 참조하여 새롭게 크롤링

2. Konlpy 또는 정규표현식(re) 라이브러리를 이용하여 위에서 수집한 감정분석에 대한 택스트 데이터에 대하여 전처리를 하고 적용한 절차와 코드를 설명하세요. 

3. 2항의 전처리 결과를 문서별 코퍼스를 토큰화하여 DTM, TF-IDF에 의한 대략적인 키워드에 대한 빈도를 분석하고 결과를 설명하세요. 

4. 위에서 처리된 내용을 기초로 로지스틱 회귀분석에 의한 감정분석을 실시하고 계수들의 웨이트를 이용한 긍정과 부정의 키워드를 시각화하여 보여주고 코드와 결과를 설명하세요.

5. 위에서 분석한 결과를 기초로 빈도수와 긍정 부정 키워드의 관계, 자료의 불균형과 모형의 예측 정확도의 관계에 대하여 위에서 분석한 사례를 들어 논하세요.   

"""
#문제의식: 영화 평가에 있어 시대에 따른 표현법의 차이에 대한 의문
#절차 1: 2019 ~ 2023 개봉영화와 2014 ~ 2018 개봉영화의 별점과 평가를 수집한다.
#절차 2: 각각의 코퍼스를 train set과 test set으로 구분한다.
#절차 3: 2019 ~ 2023 개봉영화 train set으로 훈련한 모델을 두 개의 test set에 적용하여 정확도를 비교한다
#절차 4: 2014 ~ 2013 개봉영화 train set으로 훈련한 모델을 두 개의 test set에 적용하여 정확도를 비교한다
#예측 결과1: 시의 적절한 훈련 데이터 수집의 필요성에 대해 할 수 있다.
#예측 결과2: 시대에 따른 영화 평가 방식의 표현법 차이를 수치화 할 수 있다.

'5가지 요구사항 : \n\n1. 웹크롤링을 통하여 관심주제에 대한 감정분석 데이터를 수집하고 수집한 내용과 코드 설명을 병행하여 제시하세요\n\n   *참조 : 7주차 실습 코드 3. 종합실습 코드를 참조하여 새롭게 크롤링\n\n2. Konlpy 또는 정규표현식(re) 라이브러리를 이용하여 위에서 수집한 감정분석에 대한 택스트 데이터에 대하여 전처리를 하고 적용한 절차와 코드를 설명하세요. \n\n3. 2항의 전처리 결과를 문서별 코퍼스를 토큰화하여 DTM, TF-IDF에 의한 대략적인 키워드에 대한 빈도를 분석하고 결과를 설명하세요. \n\n4. 위에서 처리된 내용을 기초로 로지스틱 회귀분석에 의한 감정분석을 실시하고 계수들의 웨이트를 이용한 긍정과 부정의 키워드를 시각화하여 보여주고 코드와 결과를 설명하세요.\n\n5. 위에서 분석한 결과를 기초로 빈도수와 긍정 부정 키워드의 관계, 자료의 불균형과 모형의 예측 정확도의 관계에 대하여 위에서 분석한 사례를 들어 논하세요.   \n\n'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import pandas as pd
import time
import requests
import json

2014~2023 사이 총 개봉영화 14683건(2023.06.06 기준)

In [3]:
#영진위 api로부터 각 년도간 영화정보 저장
start_year = 2014
end_year = 2023
api_key = '1b7d7d5da614bb2d2974f2a6ca95cded'
raw_df = pd.DataFrame(index=range(0,1), columns = ['moiveListResult','moiveList'])

for page_num in range(1,1469):
    api_url = f'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key={api_key}&openStartDt={start_year}&openEndDt={end_year}&curPage={page_num}'
    request = Request(api_url)
    response = urlopen(request).read()
    response_json = json.loads(response)
    response_df = pd.DataFrame(response_json['movieListResult']['movieList'])
    raw_df = pd.concat([raw_df, response_df], axis=0)
    print(str(page_num) + '페이지 조회했습니다')

1페이지 조회했습니다
2페이지 조회했습니다
3페이지 조회했습니다
4페이지 조회했습니다
5페이지 조회했습니다
6페이지 조회했습니다
7페이지 조회했습니다
8페이지 조회했습니다
9페이지 조회했습니다
10페이지 조회했습니다
11페이지 조회했습니다
12페이지 조회했습니다
13페이지 조회했습니다
14페이지 조회했습니다
15페이지 조회했습니다
16페이지 조회했습니다
17페이지 조회했습니다
18페이지 조회했습니다
19페이지 조회했습니다
20페이지 조회했습니다
21페이지 조회했습니다
22페이지 조회했습니다
23페이지 조회했습니다
24페이지 조회했습니다
25페이지 조회했습니다
26페이지 조회했습니다
27페이지 조회했습니다
28페이지 조회했습니다
29페이지 조회했습니다
30페이지 조회했습니다
31페이지 조회했습니다
32페이지 조회했습니다
33페이지 조회했습니다
34페이지 조회했습니다
35페이지 조회했습니다
36페이지 조회했습니다
37페이지 조회했습니다
38페이지 조회했습니다
39페이지 조회했습니다
40페이지 조회했습니다
41페이지 조회했습니다
42페이지 조회했습니다
43페이지 조회했습니다
44페이지 조회했습니다
45페이지 조회했습니다
46페이지 조회했습니다
47페이지 조회했습니다
48페이지 조회했습니다
49페이지 조회했습니다
50페이지 조회했습니다
51페이지 조회했습니다
52페이지 조회했습니다
53페이지 조회했습니다
54페이지 조회했습니다
55페이지 조회했습니다
56페이지 조회했습니다
57페이지 조회했습니다
58페이지 조회했습니다
59페이지 조회했습니다
60페이지 조회했습니다
61페이지 조회했습니다
62페이지 조회했습니다
63페이지 조회했습니다
64페이지 조회했습니다
65페이지 조회했습니다
66페이지 조회했습니다
67페이지 조회했습니다
68페이지 조회했습니다
69페이지 조회했습니다
70페이지 조회했습니다
71페이지 조회했습니다
72페이지 조회했습니다
73페이지 조회했습니다
74페이지 조회했습니다
75페이지 조회했습니다
76페이지 조회했습니다
77페이지 조회했습니다
78페이지 조회

594페이지 조회했습니다
595페이지 조회했습니다
596페이지 조회했습니다
597페이지 조회했습니다
598페이지 조회했습니다
599페이지 조회했습니다
600페이지 조회했습니다
601페이지 조회했습니다
602페이지 조회했습니다
603페이지 조회했습니다
604페이지 조회했습니다
605페이지 조회했습니다
606페이지 조회했습니다
607페이지 조회했습니다
608페이지 조회했습니다
609페이지 조회했습니다
610페이지 조회했습니다
611페이지 조회했습니다
612페이지 조회했습니다
613페이지 조회했습니다
614페이지 조회했습니다
615페이지 조회했습니다
616페이지 조회했습니다
617페이지 조회했습니다
618페이지 조회했습니다
619페이지 조회했습니다
620페이지 조회했습니다
621페이지 조회했습니다
622페이지 조회했습니다
623페이지 조회했습니다
624페이지 조회했습니다
625페이지 조회했습니다
626페이지 조회했습니다
627페이지 조회했습니다
628페이지 조회했습니다
629페이지 조회했습니다
630페이지 조회했습니다
631페이지 조회했습니다
632페이지 조회했습니다
633페이지 조회했습니다
634페이지 조회했습니다
635페이지 조회했습니다
636페이지 조회했습니다
637페이지 조회했습니다
638페이지 조회했습니다
639페이지 조회했습니다
640페이지 조회했습니다
641페이지 조회했습니다
642페이지 조회했습니다
643페이지 조회했습니다
644페이지 조회했습니다
645페이지 조회했습니다
646페이지 조회했습니다
647페이지 조회했습니다
648페이지 조회했습니다
649페이지 조회했습니다
650페이지 조회했습니다
651페이지 조회했습니다
652페이지 조회했습니다
653페이지 조회했습니다
654페이지 조회했습니다
655페이지 조회했습니다
656페이지 조회했습니다
657페이지 조회했습니다
658페이지 조회했습니다
659페이지 조회했습니다
660페이지 조회했습니다
661페이지 조회했습니다
662페이지 조회했습니다
663페이지 조회했습니다
664페이지 조회했습니다
665페이지

1168페이지 조회했습니다
1169페이지 조회했습니다
1170페이지 조회했습니다
1171페이지 조회했습니다
1172페이지 조회했습니다
1173페이지 조회했습니다
1174페이지 조회했습니다
1175페이지 조회했습니다
1176페이지 조회했습니다
1177페이지 조회했습니다
1178페이지 조회했습니다
1179페이지 조회했습니다
1180페이지 조회했습니다
1181페이지 조회했습니다
1182페이지 조회했습니다
1183페이지 조회했습니다
1184페이지 조회했습니다
1185페이지 조회했습니다
1186페이지 조회했습니다
1187페이지 조회했습니다
1188페이지 조회했습니다
1189페이지 조회했습니다
1190페이지 조회했습니다
1191페이지 조회했습니다
1192페이지 조회했습니다
1193페이지 조회했습니다
1194페이지 조회했습니다
1195페이지 조회했습니다
1196페이지 조회했습니다
1197페이지 조회했습니다
1198페이지 조회했습니다
1199페이지 조회했습니다
1200페이지 조회했습니다
1201페이지 조회했습니다
1202페이지 조회했습니다
1203페이지 조회했습니다
1204페이지 조회했습니다
1205페이지 조회했습니다
1206페이지 조회했습니다
1207페이지 조회했습니다
1208페이지 조회했습니다
1209페이지 조회했습니다
1210페이지 조회했습니다
1211페이지 조회했습니다
1212페이지 조회했습니다
1213페이지 조회했습니다
1214페이지 조회했습니다
1215페이지 조회했습니다
1216페이지 조회했습니다
1217페이지 조회했습니다
1218페이지 조회했습니다
1219페이지 조회했습니다
1220페이지 조회했습니다
1221페이지 조회했습니다
1222페이지 조회했습니다
1223페이지 조회했습니다
1224페이지 조회했습니다
1225페이지 조회했습니다
1226페이지 조회했습니다
1227페이지 조회했습니다
1228페이지 조회했습니다
1229페이지 조회했습니다
1230페이지 조회했습니다
1231페이지 조회했습니다
1232페이지 조회했습니다
1233페이지 조회했습니다
1234페이지 조회

In [4]:
#DF 변환 확인 
raw_df.head()

,moiveListResult,moiveList,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,20231971,플래시,The Flash,2023,20230614,장편,개봉예정,미국,액션,미국,액션,[{'peopleNm': '안드레스 무시에티'}],[]
1,NaN,NaN,20231923,엔니오: 더 마에스트로,Ennio,2021,20230705,장편,개봉예정,이탈리아,다큐멘터리,이탈리아,다큐멘터리,[{'peopleNm': '쥬세페 토르나토레'}],[]
2,NaN,NaN,20232075,명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인,Detective Conan: The Story of Ai Haibara: Blac...,2023,20230623,장편,개봉예정,일본,애니메이션,일본,애니메이션,[],[]
3,NaN,NaN,20232051,뮤지컬 청춘소음,,2023,20230607,장편,개봉예정,한국,공연,한국,공연,[],"[{'companyCd': '20110019', 'companyNm': '한국문화예..."


In [5]:
raw_df = raw_df.drop(raw_df.index[0])

api 명세서{movieNm: 영화명, openDt:개봉일시}

In [6]:
df = raw_df[['movieNm','openDt']].copy()
df['openDt'] = df['openDt'].str[:4].astype(int)

In [7]:
df.info()
print(df.head())
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13212 entries, 1 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieNm  13212 non-null  object
 1   openDt   13212 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 309.7+ KB
                            movieNm  openDt
1                      엔니오: 더 마에스트로    2023
2  명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인    2023
3                          뮤지컬 청춘소음    2023
4                        김재훈의 피.엔.오    2023
5                             배우의 꿈    2023
             movieNm  openDt
5                 코멧    2015
6             뷰티풀 라이    2015
7             나쁜 녀석들    2015
8  울언니: 성상납의 비밀 무삭제판    2015
9          몽골리안 프린세스    2015


In [8]:
#concat으로 인한 index 중복지정 문제가 있어 index 처음부터 재지정
df.reset_index(inplace=True,drop = True)
df.info()
print(df.head())
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13212 entries, 0 to 13211
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieNm  13212 non-null  object
 1   openDt   13212 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 206.6+ KB
                            movieNm  openDt
0                      엔니오: 더 마에스트로    2023
1  명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인    2023
2                          뮤지컬 청춘소음    2023
3                        김재훈의 피.엔.오    2023
4                             배우의 꿈    2023
                 movieNm  openDt
13207                 코멧    2015
13208             뷰티풀 라이    2015
13209             나쁜 녀석들    2015
13210  울언니: 성상납의 비밀 무삭제판    2015
13211          몽골리안 프린세스    2015


In [9]:
#백업
df.to_csv('movie_data_frame.csv')

In [10]:
df_smaller_than_2019 = df[df['openDt'] < 2019]
df_greater_or_equal_to_2019 = df[df['openDt'] >= 2019]

In [11]:
df_smaller_than_2019.info()
df_greater_or_equal_to_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5878 entries, 12 to 13211
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieNm  5878 non-null   object
 1   openDt   5878 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 137.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7334 entries, 0 to 7688
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieNm  7334 non-null   object
 1   openDt   7334 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 171.9+ KB


In [12]:
#각 데이터프레임에셔 영화명 추출
older_movie_names = df_smaller_than_2019['movieNm'].tolist()
newer_movie_names = df_greater_or_equal_to_2019['movieNm'].tolist()

In [13]:
#각 리스트 내용 확인
print(older_movie_names[:5])
print(newer_movie_names[:5])

['앙상블', '맘마미아!2', '연평해전', '아일라', '순응자']
['엔니오: 더 마에스트로', '명탐정코난: 하이바라 아이 이야기 ~흑철의 미스터리 트레인', '뮤지컬 청춘소음', '김재훈의 피.엔.오', '배우의 꿈']


for moive_name in older_movie_names
# 드라이버 사용해서 크롬 열기
driver = webdriver.Chrome('chromedriver_mac_arm64/chromedriver')
driver.get(f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=영화+{movie_names}+평점')
time.sleep(15)
driver.execute_script("window.scrollTo(0, 800)")
time.sleep(2)

# 스크롤용 경로 생성
x_path = "/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"
to_scroll = driver.find_element_by_xpath(x_path)

# 별점과 코멘트 초기화
comments = []
ratings = []
times = 0
counts = 0

while True:
    times += 1
    all_contents = driver.find_elements(By.CSS_SELECTOR, "div.lego_review_list._scroller")

    flag = False  # Flag variable to track if break has already been executed
    
    for content in all_contents:
        li_comments = content.find_elements(By.CSS_SELECTOR, "span.desc._text")
        li_ratings = content.find_elements(By.CSS_SELECTOR, "div.area_text_box")
        for li_comment in li_comments[counts:]:
            comments.append(li_comment.text)

        for li_rating in li_ratings[counts:]:
            rating_value = li_rating.get_attribute("textContent").replace("별점(10점 만점 중)", "").strip()
            ratings.append(int(rating_value))
            
    if len(comments) == counts:
        flag = True  # Set flag to True if the break statement is executed
        break
    
    if flag:
        print('종료')
        break
    counts = len(comments)
    print(str(movie_name) + ' ' +str(times) + ' 회 실행했습니다')

    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", to_scroll)
    time.sleep(3)
    
driver.quit()

print(comments)
print(ratings)
print(len(comments))
print(len(ratings))


data = []
times=0
for movie_name in older_movie_names:
    # 드라이버 사용해서 크롬 열기
    driver = webdriver.Chrome('chromedriver_mac_arm64/chromedriver')
    driver.get(f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=영화+{movie_name}+평점')
    time.sleep(15)
    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(2)

    # 스크롤용 경로 생성
    x_path = "/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"
    to_scroll = driver.find_element_by_xpath(x_path)

    # 별점과 코멘트 초기화
    counts = 0
    movie_comments = []
    movie_ratings = []

    while True:
        try:
            all_contents = driver.find_elements(By.CSS_SELECTOR, "div.lego_review_list._scroller")

            flag = False  # Flag variable to track if break has already been executed

            for content in all_contents:
                li_comments = content.find_elements(By.CSS_SELECTOR, "span.desc._text")
                li_ratings = content.find_elements(By.CSS_SELECTOR, "div.area_text_box")
                for li_comment in li_comments[counts:]:
                    movie_comments.append(li_comment.text)

                for li_rating in li_ratings[counts:]:
                    rating_value = li_rating.get_attribute("textContent").replace("별점(10점 만점 중)", "").strip()
                    movie_ratings.append(int(rating_value))

            if len(movie_comments) == counts:
                flag = True  # Set flag to True if the break statement is executed
                break

            if flag:
                print('종료')
                break
            counts = len(movie_comments)
            times += 1
            print(str(movie_name) + str(times) + ' 회 실행했습니다')

            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", to_scroll)
            time.sleep(3)
       
        driver.quit()
        except:
            continue
    # 영화정보로 딕셔너리 생성
    movie_data = {
        'movieName': [movie_name] * len(movie_comments),
        'comments': movie_comments,
        'ratings': movie_ratings
    }

    # data 리스트에 정보추가
    data.extend(pd.DataFrame(movie_data).to_dict('records'))

# Create a DataFrame from the data list
older_movie_df = pd.DataFrame(data)

In [21]:
data = []
times = 0
for movie_name in older_movie_names:
    # 드라이버 사용해서 크롬 열기
    driver = webdriver.Chrome('chromedriver_mac_arm64/chromedriver')
    driver.get(f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=영화+{movie_name}+평점')
    time.sleep(15)
    driver.execute_script("window.scrollTo(0, 800)")
    time.sleep(2)

    # 스크롤용 경로 생성
    x_path = "/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"
    try:
        to_scroll = driver.find_element_by_xpath(x_path)
        
    except Exception as e:
        print(f"An error occurred: {str(e)} for {movie_name}")
        continue
        
    # 별점과 코멘트 초기화
    counts = 0
    movie_comments = []
    movie_ratings = []

    while True:

        all_contents = driver.find_elements(By.CSS_SELECTOR, "div.lego_review_list._scroller")

        flag = False  # Flag variable to track if break has already been executed

        for content in all_contents:
            li_comments = content.find_elements(By.CSS_SELECTOR, "span.desc._text")
            li_ratings = content.find_elements(By.CSS_SELECTOR, "div.area_text_box")
            for li_comment in li_comments[counts:]:
                movie_comments.append(li_comment.text)

            for li_rating in li_ratings[counts:]:
                rating_value = li_rating.get_attribute("textContent").replace("별점(10점 만점 중)", "").strip()
                movie_ratings.append(int(rating_value))

        if len(movie_comments) == counts:
            flag = True  # Set flag to True if the break statement is executed
            break

        if flag:
            print('종료')
            break
        counts = len(movie_comments)
        times += 1
        print(str(movie_name) + str(times) + ' 회 실행했습니다')

        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", to_scroll)
        time.sleep(3)

        
    if len(movie_comments) == 0:
        continue
    driver.quit()
    # 영화정보로 딕셔너리 생성
    movie_data = {
        'movieName': [movie_name] * len(movie_comments),
        'comments': movie_comments,
        'ratings': movie_ratings
    }

    # data 리스트에 정보추가
    data.extend(pd.DataFrame(movie_data).to_dict('records'))

# Create a DataFrame from the data list
older_movie_df = pd.DataFrame(data)


앙상블1 회 실행했습니다
앙상블2 회 실행했습니다
맘마미아!23 회 실행했습니다
맘마미아!24 회 실행했습니다
맘마미아!25 회 실행했습니다
맘마미아!26 회 실행했습니다
맘마미아!27 회 실행했습니다
맘마미아!28 회 실행했습니다
맘마미아!29 회 실행했습니다
맘마미아!210 회 실행했습니다
맘마미아!211 회 실행했습니다
맘마미아!212 회 실행했습니다
맘마미아!213 회 실행했습니다
맘마미아!214 회 실행했습니다
맘마미아!215 회 실행했습니다
맘마미아!216 회 실행했습니다
맘마미아!217 회 실행했습니다
맘마미아!218 회 실행했습니다
맘마미아!219 회 실행했습니다
맘마미아!220 회 실행했습니다
맘마미아!221 회 실행했습니다
맘마미아!222 회 실행했습니다
맘마미아!223 회 실행했습니다
맘마미아!224 회 실행했습니다
맘마미아!225 회 실행했습니다
맘마미아!226 회 실행했습니다
맘마미아!227 회 실행했습니다
맘마미아!228 회 실행했습니다
맘마미아!229 회 실행했습니다
맘마미아!230 회 실행했습니다
맘마미아!231 회 실행했습니다
맘마미아!232 회 실행했습니다
맘마미아!233 회 실행했습니다
연평해전34 회 실행했습니다
연평해전35 회 실행했습니다
연평해전36 회 실행했습니다
연평해전37 회 실행했습니다
연평해전38 회 실행했습니다
연평해전39 회 실행했습니다
연평해전40 회 실행했습니다
연평해전41 회 실행했습니다
연평해전42 회 실행했습니다
연평해전43 회 실행했습니다
연평해전44 회 실행했습니다
연평해전45 회 실행했습니다
연평해전46 회 실행했습니다
연평해전47 회 실행했습니다
연평해전48 회 실행했습니다
연평해전49 회 실행했습니다
연평해전50 회 실행했습니다
연평해전51 회 실행했습니다
연평해전52 회 실행했습니다
연평해전53 회 실행했습니다
연평해전54 회 실행했습니다
연평해전55 회 실행했습니다
연평해전56 회 실행했습니다
연평해전57 회 실행했습니다
연평해전58 회 실행했습니다
연평해전59 회 실행했습니다
연평해전6

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 가정부 : 맛있는 서비스
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 룸싸롱 맛있는 서비스 무삭제
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 내 아내의 언니 2 무삭제
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 내 아내의 언니 2
An error occurred: Message: no such element: Unable to l

달라스 바이어스 클럽632 회 실행했습니다
달라스 바이어스 클럽633 회 실행했습니다
달라스 바이어스 클럽634 회 실행했습니다
달라스 바이어스 클럽635 회 실행했습니다
달라스 바이어스 클럽636 회 실행했습니다
달라스 바이어스 클럽637 회 실행했습니다
캐롤638 회 실행했습니다
캐롤639 회 실행했습니다
캐롤640 회 실행했습니다
캐롤641 회 실행했습니다
캐롤642 회 실행했습니다
캐롤643 회 실행했습니다
캐롤644 회 실행했습니다
캐롤645 회 실행했습니다
캐롤646 회 실행했습니다
캐롤647 회 실행했습니다
캐롤648 회 실행했습니다
캐롤649 회 실행했습니다
캐롤650 회 실행했습니다
캐롤651 회 실행했습니다
캐롤652 회 실행했습니다
캐롤653 회 실행했습니다
캐롤654 회 실행했습니다
캐롤655 회 실행했습니다
캐롤656 회 실행했습니다
캐롤657 회 실행했습니다
캐롤658 회 실행했습니다
캐롤659 회 실행했습니다
캐롤660 회 실행했습니다
캐롤661 회 실행했습니다
캐롤662 회 실행했습니다
캐롤663 회 실행했습니다
캐롤664 회 실행했습니다
캐롤665 회 실행했습니다
캐롤666 회 실행했습니다
캐롤667 회 실행했습니다
캐롤668 회 실행했습니다
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 환절기
돌아와요 부산항애(愛)669 회 실행했습니다
돌아와요 부산항애(愛)670 회 실행했습니다
돌아와요 부산항애(愛)671 회 실행했습니다
돌아와요 부산항애(愛)672 회 실행했습니다
사도673 회 실행했습니다
사도674 회 실행했습니다
사도675 회 실행했습니다
사도676 회 실행했습니다
사도677

그물1036 회 실행했습니다
아쿠아맨1037 회 실행했습니다
아쿠아맨1038 회 실행했습니다
아쿠아맨1039 회 실행했습니다
아쿠아맨1040 회 실행했습니다
아쿠아맨1041 회 실행했습니다
아쿠아맨1042 회 실행했습니다
아쿠아맨1043 회 실행했습니다
아쿠아맨1044 회 실행했습니다
아쿠아맨1045 회 실행했습니다
아쿠아맨1046 회 실행했습니다
아쿠아맨1047 회 실행했습니다
아쿠아맨1048 회 실행했습니다
아쿠아맨1049 회 실행했습니다
아쿠아맨1050 회 실행했습니다
아쿠아맨1051 회 실행했습니다
아쿠아맨1052 회 실행했습니다
아쿠아맨1053 회 실행했습니다
아쿠아맨1054 회 실행했습니다
아쿠아맨1055 회 실행했습니다
아쿠아맨1056 회 실행했습니다
아쿠아맨1057 회 실행했습니다
아쿠아맨1058 회 실행했습니다
아쿠아맨1059 회 실행했습니다
아쿠아맨1060 회 실행했습니다
아쿠아맨1061 회 실행했습니다
아쿠아맨1062 회 실행했습니다
아쿠아맨1063 회 실행했습니다
아쿠아맨1064 회 실행했습니다
아쿠아맨1065 회 실행했습니다
아쿠아맨1066 회 실행했습니다
아쿠아맨1067 회 실행했습니다
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.90)
 for 신비한 동물사전
한공주1068 회 실행했습니다
한공주1069 회 실행했습니다
한공주1070 회 실행했습니다
한공주1071 회 실행했습니다
한공주1072 회 실행했습니다
한공주1073 회 실행했습니다
한공주1074 회 실행했습니다
한공주1075 회 실행했습니다
한공주1076 회 실행했습니다
한공주1077 회 실행했습니다
한공주1078 회 실행했습니다
한공주1079 

위플래쉬1394 회 실행했습니다
위플래쉬1395 회 실행했습니다
위플래쉬1396 회 실행했습니다
위플래쉬1397 회 실행했습니다
위플래쉬1398 회 실행했습니다
위플래쉬1399 회 실행했습니다
위플래쉬1400 회 실행했습니다
위플래쉬1401 회 실행했습니다
위플래쉬1402 회 실행했습니다
위플래쉬1403 회 실행했습니다
위플래쉬1404 회 실행했습니다
위플래쉬1405 회 실행했습니다
위플래쉬1406 회 실행했습니다
위플래쉬1407 회 실행했습니다
위플래쉬1408 회 실행했습니다
위플래쉬1409 회 실행했습니다
위플래쉬1410 회 실행했습니다
위플래쉬1411 회 실행했습니다
예스 평창!1412 회 실행했습니다
예스 평창!1413 회 실행했습니다
예스 평창!1414 회 실행했습니다
예스 평창!1415 회 실행했습니다
내 이름은 꾸제트1416 회 실행했습니다
내 이름은 꾸제트1417 회 실행했습니다
내 이름은 꾸제트1418 회 실행했습니다
내 이름은 꾸제트1419 회 실행했습니다
내 이름은 꾸제트1420 회 실행했습니다
내 이름은 꾸제트1421 회 실행했습니다
내 이름은 꾸제트1422 회 실행했습니다
내 이름은 꾸제트1423 회 실행했습니다
내 이름은 꾸제트1424 회 실행했습니다
구름의 저편, 약속의 장소1425 회 실행했습니다
구름의 저편, 약속의 장소1426 회 실행했습니다
구름의 저편, 약속의 장소1427 회 실행했습니다
구름의 저편, 약속의 장소1428 회 실행했습니다
구름의 저편, 약속의 장소1429 회 실행했습니다
너의 이름은.1430 회 실행했습니다
너의 이름은.1431 회 실행했습니다
너의 이름은.1432 회 실행했습니다
너의 이름은.1433 회 실행했습니다
너의 이름은.1434 회 실행했습니다
너의 이름은.1435 회 실행했습니다
너의 이름은.1436 회 실행했습니다
너의 이름은.1437 회 실행했습니다
너의 이름은.1438 회 실행했습니다
너의 이름은.1439 회 실행했습니다
너의 이름은.1440 회 실행했습니다
너의 이름은.1441 회 실행했습

...

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 새엄마 2 무삭제
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 내 친구 엄마 무삭제
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 내 여자의 정사: 감독판
명탐정 코난: 순흑의 악몽7893 회 실행했습니다
명탐정 코난: 순흑의 악몽7894 회 실행했습니다
명탐정 코난: 순흑의 악몽7895 회 실행했습니다
명탐정 코난: 순흑의 악몽7896 회 실행했습니다
명탐정 코난: 순흑의 악몽7897 회 실행했습니다
명탐정 코난: 순흑의 악몽7898 회 실행했습니다
명탐정 코난: 순흑의 악몽7899 회 실행했습니다
명탐정 코난: 순흑의 악몽7900 회 실행했습니다
명탐정 코난: 순흑의 악몽7901 회 실행했습니다
명탐정 코난: 순흑의 악몽7902 회 실행했습니다
명탐정 코난: 순흑의 악몽7

극장판 포켓몬스터DP - 디아루가 VS 펄기아 VS 다크라이8030 회 실행했습니다
극장판 포켓몬스터DP - 디아루가 VS 펄기아 VS 다크라이8031 회 실행했습니다
극장판 포켓몬스터DP - 디아루가 VS 펄기아 VS 다크라이8032 회 실행했습니다
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 에너미라인 1994 대전투
불한당: 나쁜 놈들의 세상8033 회 실행했습니다
불한당: 나쁜 놈들의 세상8034 회 실행했습니다
불한당: 나쁜 놈들의 세상8035 회 실행했습니다
불한당: 나쁜 놈들의 세상8036 회 실행했습니다
불한당: 나쁜 놈들의 세상8037 회 실행했습니다
불한당: 나쁜 놈들의 세상8038 회 실행했습니다
불한당: 나쁜 놈들의 세상8039 회 실행했습니다
불한당: 나쁜 놈들의 세상8040 회 실행했습니다
불한당: 나쁜 놈들의 세상8041 회 실행했습니다
불한당: 나쁜 놈들의 세상8042 회 실행했습니다
불한당: 나쁜 놈들의 세상8043 회 실행했습니다
불한당: 나쁜 놈들의 세상8044 회 실행했습니다
불한당: 나쁜 놈들의 세상8045 회 실행했습니다
불한당: 나쁜 놈들의 세상8046 회 실행했습니다
불한당: 나쁜 놈들의 세상8047 회 실행했습니다
불한당: 나쁜 놈들의 세상8048 회 실행했습니다
불한당: 나쁜 놈들의 세상8049 회 실행했습니다
불한당: 나쁜 놈들의 세상8050 회 실행했습니다
불한당: 나쁜 놈들의 세상8051 회 실행했습니다
불한당: 나쁜 놈들의 세상8052 회 실행했습니다
불한당: 나쁜 놈들의 세상8053 회 실행했습니다
불한당: 나쁜 놈들의 세상8054 회 실행했습니다
불한당: 

남자와 여자8237 회 실행했습니다
남자와 여자8238 회 실행했습니다
남자와 여자8239 회 실행했습니다
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 착한 장모들
An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div[2]/div/div[2]/div[6]"}
  (Session info: chrome=114.0.5735.106)
 for 사촌 여동생들: 금단의 사랑
라스트 미션8240 회 실행했습니다
라스트 미션8241 회 실행했습니다
라스트 미션8242 회 실행했습니다
라스트 미션8243 회 실행했습니다
라스트 미션8244 회 실행했습니다
라스트 미션8245 회 실행했습니다
라스트 미션8246 회 실행했습니다
라스트 미션8247 회 실행했습니다
라스트 미션8248 회 실행했습니다
라스트 미션8249 회 실행했습니다
라스트 미션8250 회 실행했습니다
라스트 미션8251 회 실행했습니다
라스트 미션8252 회 실행했습니다
라스트 미션8253 회 실행했습니다
라스트 미션8254 회 실행했습니다
라스트 미션8255 회 실행했습니다
라스트 미션8256 회 실행했습니다
라스트 미션8257 회 실행했습니다
라스트 미션8258 회 실행했습니다
라스트 미션8259 회 실행했습니다
An error occurred: Message: no such element: Unable to locate element: 

나의 산티아고8348 회 실행했습니다
나의 산티아고8349 회 실행했습니다
나의 산티아고8350 회 실행했습니다
나의 산티아고8351 회 실행했습니다
나의 산티아고8352 회 실행했습니다
나의 산티아고8353 회 실행했습니다
나의 산티아고8354 회 실행했습니다
나의 산티아고8355 회 실행했습니다
나의 산티아고8356 회 실행했습니다
나의 산티아고8357 회 실행했습니다
나의 산티아고8358 회 실행했습니다
나의 산티아고8359 회 실행했습니다
나의 산티아고8360 회 실행했습니다
나의 산티아고8361 회 실행했습니다
나의 산티아고8362 회 실행했습니다
나의 산티아고8363 회 실행했습니다
나의 산티아고8364 회 실행했습니다
나의 산티아고8365 회 실행했습니다
나의 산티아고8366 회 실행했습니다
나의 산티아고8367 회 실행했습니다
나의 산티아고8368 회 실행했습니다
나의 산티아고8369 회 실행했습니다
나의 산티아고8370 회 실행했습니다
나의 산티아고8371 회 실행했습니다
나의 산티아고8372 회 실행했습니다
영혼의 순례길8373 회 실행했습니다
바라나시8374 회 실행했습니다
누에치던 방8375 회 실행했습니다
누에치던 방8376 회 실행했습니다
누에치던 방8377 회 실행했습니다
살인소설8378 회 실행했습니다
샷 콜러8379 회 실행했습니다
트립 투 스페인8380 회 실행했습니다
트립 투 스페인8381 회 실행했습니다
트립 투 스페인8382 회 실행했습니다
트립 투 스페인8383 회 실행했습니다
트립 투 스페인8384 회 실행했습니다
클레어의 카메라8385 회 실행했습니다
문호 스트레이독스 [데드 애플]8386 회 실행했습니다
문호 스트레이독스 [데드 애플]8387 회 실행했습니다
문호 스트레이독스 [데드 애플]8388 회 실행했습니다
문호 스트레이독스 [데드 애플]8389 회 실행했습니다
문호 스트레이독스 [데드 애플]8390 회 실행했습니다
문호 스트레이독스 [데드 애플]8391 회 실행했습니다
문호 스트레이독스 [데드 애플]8392 

KeyboardInterrupt: 

In [22]:
stop_words = []
stop_words_txt = 'stopword.txt'
with open(stop_words_txt) as f:
    lines = f.readlines()

stop_words = [line.rstrip('\n') for line in lines]
print(stop_words)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',